In [20]:
import speech_recognition as sr
import pandas as pd
import re
from fuzzywuzzy import fuzz

from googletrans import Translator
from gtts import gTTS
from playsound import playsound

# Load the training data
training_data = pd.read_csv("BigBasket Products.csv")

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Function to find the most similar label to the input keywords
def find_similar_label(input_keywords, labels):
    similarities = {}
    for label in labels:
        label_keywords = preprocess_text(label).split()
        common_keywords = set(input_keywords) & set(label_keywords)
        if common_keywords:
            similarity_score = fuzz.ratio(' '.join(input_keywords), ' '.join(label_keywords))
            similarities[label] = similarity_score
    if not similarities:
        return None
    most_similar_label = max(similarities, key=similarities.get)
    return most_similar_label

# Function to get data corresponding to a label
def get_data_for_label(label, training_data):
    data = training_data.loc[training_data['Label'] == label]['Data']
    return [d.strip() for d in data.values[0].split(',')]

# Function to get amount for a label
def get_amount_for_label(label, training_data):
    return training_data.loc[training_data['Label'] == label]['Amount'].values[0]

# Function for speech recognition
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        print("Recognizing...")
        text = recognizer.recognize_google(audio, language='en')
        print("Recognized text:", text)
        return text
    except sr.UnknownValueError:
        print("Could not understand audio.")
        return ""
    except sr.RequestError as e:
        print("Error with the service; {0}".format(e))
        return ""

# Main function
def main():
    total_amount = 0
    identified_labels = set()
    while True:
        print("Speak something: ")
        original_text = recognize_speech()
        
        if original_text.lower() == "end shopping":
            break
        
        input_string = original_text
        # Preprocess the input string to extract keywords after conjunction words
        input_keywords = preprocess_text(input_string).split()
        conjunction_indices = [i for i, word in enumerate(input_keywords) if word in ['and', 'but']]
        keyword_groups = []
        start_idx = 0
        for idx in conjunction_indices:
            keyword_groups.append(input_keywords[start_idx:idx])
            start_idx = idx + 1
        keyword_groups.append(input_keywords[start_idx:])
        
        for keywords in keyword_groups:
            similar_label = find_similar_label(keywords, training_data['Label'])
            if similar_label in identified_labels:
                print(similar_label + " is already added to your cart.")
            elif similar_label:
                identified_labels.add(similar_label)
                data = get_data_for_label(similar_label, training_data)
                amount = get_amount_for_label(similar_label, training_data)
                total_amount += amount
                print(similar_label + " Ingredients:")
                for d in data:
                    print(f"{d.ljust(15)} = Rs.{amount}")
                print()
    print("Total Bill Amount: Rs." + str(total_amount))
    print("!!!! Note: HAPPY SHOPPING !!!!")
    
if __name__ == "__main__":
    main()
    

Speak something: 
Listening...
Recognizing...
Recognized text: I want Dosa
DOSA Ingredients:
gram flour 500grms = Rs.40 = Rs.145
rice flour 1kg = Rs.60 = Rs.145
urad dhal 1kg = Rs.45 = Rs.145

Speak something: 
Listening...
Recognizing...
Could not understand audio.
Speak something: 
Listening...
Recognizing...
Recognized text: Technology
Speak something: 
Listening...
Recognizing...
Recognized text: I want Dosa and sambar
DOSA is already added to your cart.
SAMBAR Ingredients:
toor dal 1kg = Rs.40 = Rs.110
tomatoes 500grms = Rs.25 = Rs.110
tamarind 100grms = Rs.15 = Rs.110
sambar powder 1pac = Rs.30 = Rs.110

Speak something: 
Listening...
Recognizing...
Recognized text: already added to the card sambar ingredients
SAMBAR is already added to your cart.
Speak something: 
Listening...
Recognizing...
Recognized text: end shopping
Total Bill Amount: Rs.255
!!!! Note: HAPPY SHOPPING !!!!
